In [44]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import warnings
from datetime import datetime

# 불필요한 경고 메시지를 숨깁니다.
warnings.filterwarnings('ignore')

In [45]:
def predict(lotto_history_data, n_estimators=100, random_state=350, verbose=0):
    # 데이터를 Pandas DataFrame으로 변환합니다.
    df = pd.DataFrame(lotto_history_data, columns=['회차', '번호1', '번호2', '번호3', '번호4', '번호5', '번호6'])
    df = df.sort_values(by='회차').reset_index(drop=True)

    if verbose > 0:
        print("--- 로또 당첨 번호 이력 ---")
        print(df)

    # 기계 학습을 위한 데이터 준비
    # 각 회차의 당첨 번호(X)와 바로 다음 회차의 당첨 번호(y)를 사용합니다.
    X = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']]
    y = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].shift(-1)

    # 마지막 행은 다음 회차 데이터가 없으므로 제거합니다.
    X = X[:-1]
    y = y.dropna()
    
    if verbose > 0:
        print("\n--- 훈련 데이터 (X) ---")
        print(X.head())
        print("\n--- 정답 데이터 (y) ---")
        print(y.head())

    # 머신러닝 모델 선택 및 훈련
    # 랜덤 포레스트 회귀 모델을 사용합니다.
    # the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
    # the n_estimators parameter specifies the number of decision trees in the forest. 
    # model = RandomForestRegressor(n_estimators=50, random_state=1000) # n_estimators: 만들 트리의 개수 => 3개 일치
    model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state) # n_estimators: 만들 트리의 개수
    model.fit(X, y)

    # 예측할 회차의 이전 회차 데이터 (1173회차)
    last_draw = df.iloc[-1][['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.reshape(1, -1)

    if verbose > 0:
        print(f"\n--- {df.iloc[-1]['회차']}회차 데이터로 다음 회차 예측 ---")
        print(last_draw)

    # 다음 회차(1174회) 번호 예측
    predicted_numbers_set = []
    trial = 5
    for i in range(trial):
        predicted_numbers_float = model.predict(last_draw)

        # 예측된 번호 처리
        # 1. 소수점을 반올림하여 정수로 만듭니다.
        # 2. 1~45 사이의 값으로 보정합니다.
        # 3. 중복된 번호를 제거하고 6개를 선택합니다.
        predicted_numbers = set()
        for num in predicted_numbers_float[0]:
            # 반올림하여 정수로 변환
            int_num = int(round(num))
    
            # 1보다 작으면 1로, 45보다 크면 45로 보정
            if int_num < 1:
                int_num = 1
            elif int_num > 45:
                int_num = 45
            predicted_numbers.add(int_num)

        # 중복 제거 후 6개가 안되면, 부족한 만큼 다른 번호로 채웁니다.
        # (여기서는 가장 빈도가 높은 번호들 중 예측되지 않은 번호를 추가하는 방식을 사용)
        if len(predicted_numbers) < 6:
            all_numbers = df[['번호1', '번호2', '번호3', '번호4', '번호5', '번호6']].values.flatten()
            counts = pd.Series(all_numbers).value_counts()
    
            extra_needed = 6 - len(predicted_numbers)
            for num in counts.index:
                if extra_needed == 0:
                    break
                if num not in predicted_numbers:
                    predicted_numbers.add(num)
                    extra_needed -= 1
        # 최종 예측 번호를 정렬하여 출력
        final_prediction = sorted(list(predicted_numbers))[:6]
        predicted_numbers_set.append(final_prediction)
    return predicted_numbers_set


def test_prediction(round_nums,
                    actual_numbers,
                    bonus,
                    predicted_numbers_set,
                    n_estimators=500,
                    random_state=500,
                    only_one=True,
                    verbose=0):
    if verbose>0:
        print("======================================")
        print(f"\n--- 참고: {round_nums}회 실제 당첨 번호 ---")
        print(f"당첨번호: {actual_numbers}, 보너스: {bonus}")

    matched_len = 0

    for final_prediction in predicted_numbers_set:
        # 예측 결과와 실제 결과 비교
        matching_numbers = set(final_prediction).intersection(set(actual_numbers))
        if verbose > 0:
            print(f"\n--- 예측과 실제 결과 비교 ---")
            print(f"일치하는 번호: {sorted(list(matching_numbers))}")
            print(f"일치 개수: {len(matching_numbers)}")
            print(F"예측 번호: {final_prediction}")
            print(f"n_estimators: {n_estimators}")
            print(f"random_state: {random_state}")
        matched_len = len(matching_numbers)
        if only_one:
            break
    return matched_len

In [46]:
# 실제 1174회 당첨 번호
actual_numbers = [
    [1, 2, 3, 4, 5, 6],
    [8, 11, 14, 17, 36, 39],
    [1, 5, 18, 20, 30, 35],
    [7, 9, 24, 40, 42, 44],
]
bonus = [0, 22, 22]
round_nums = [1175, 1174, 1173]
# 역대 로또 당첨 번호 데이터 (최신 회차부터 순서대로)
# 실제로는 더 많은 데이터가 필요하지만, 예시를 위해 일부만 사용합니다.
# 데이터 출처: https://data.soledot.com/lottowinnumber/fo/lottowinnumberlist.sd
lotto_history_data = [
    [1174, 8, 11, 14, 17, 36, 39],
    [1173, 1, 5, 18, 20, 30, 35],
    [1172, 7, 9, 24, 40, 42, 44],
    [1171, 3, 6, 7, 11, 12, 17],
    [1170, 3, 13, 28, 34, 38, 42],
    [1169, 5, 12, 24, 26, 39, 42],
    [1168, 9, 21, 24, 30, 33, 37],
    [1167, 8, 23, 31, 35, 39, 40],
    [1166, 14, 23, 25, 27, 29, 42],
    [1165, 6, 7, 27, 29, 38, 45],
    [1164, 17, 18, 23, 25, 38, 39],
    [1163, 2, 13, 15, 16, 33, 43],
    [1162, 20, 21, 22, 25, 28, 29],
    [1161, 2, 12, 20, 24, 34, 42],
    [1160, 7, 13, 18, 36, 39, 45],
    [1159, 3, 9, 27, 28, 38, 39],
    [1158, 21, 25, 27, 32, 37,38],
    [1157, 5, 7, 12, 20, 25, 26],
    [1156, 30, 31, 34, 39, 41, 45],
    [1155, 10, 16, 19, 27, 37, 38],
    [1154, 4, 8, 22, 26, 32, 38],
    [1153, 1, 9, 10, 13, 35, 44],
    [1152, 30, 31, 32, 35, 36, 37],
    [1151, 2, 3, 9, 15, 27, 29],
    [1150, 8, 9, 18, 35, 39, 45],
]

r_pos = [
    (0, len(lotto_history_data)-3+1),
    (1, len(lotto_history_data)-2+1),
    (2, len(lotto_history_data)-1+1),
]

**Notice**
- the random_state parameter is used to control the randomness of the algorithm, ensuring reproducibility of results. 
- the n_estimators parameter specifies the number of decision trees in the forest. This parameter is crucial as it directly impacts the model's performance and computational cost.

In [35]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=8
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 18:45:16.824615]
found =  45 1700 2 2 2
found =  45 12600 2 2 2
found =  45 14300 3 2 2
found =  45 16200 2 2 2
found =  45 32200 3 2 2
found =  45 52900 2 2 2
found =  45 75800 2 2 2
found =  45 82700 2 2 2
found =  45 83100 2 2 2
found =  45 90700 2 2 2
found =  45 91800 2 2 2
proceed [now=2025-06-06 18:46:30.913466, 1000]
found =  45 101500 2 2 2
found =  45 124400 2 2 2
found =  45 128300 3 2 2
found =  45 131900 3 2 2
found =  45 145100 3 2 2
found =  45 153700 2 2 2
found =  45 171700 2 2 2
found =  45 185100 3 2 2
found =  45 192800 2 2 2
proceed [now=2025-06-06 18:47:45.926896, 2000]
found =  45 200700 3 2 2
found =  45 203700 2 2 2
found =  45 209500 2 2 2
found =  45 210400 2 2 2
found =  45 230900 2 2 2
found =  45 236500 2 2 2
found =  45 239900 2 3 2
found =  45 242900 2 2 2
found =  45 257000 2 2 2
found =  45 265300 3 2 2
found =  45 273600 2 2 2
found =  45 278300 2 2 2
found =  45 290200 2 2 2
found =  45 294000 3 2 2
proceed [now=2025-06-06 18:49

In [36]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=7
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 18:49:14.689484]
found =  45 16600 3 2 1
found =  45 18800 2 2 1
found =  45 30500 2 2 1
found =  45 39600 3 2 1
found =  45 54600 2 2 1
found =  45 63400 3 2 1
found =  45 64000 2 2 1
found =  45 78100 3 2 1
proceed [now=2025-06-06 18:50:30.812771, 1000]
found =  45 113700 2 2 1
found =  45 133900 2 2 1
found =  45 137000 3 2 1
found =  45 152700 2 2 1
found =  45 165100 2 2 1
found =  45 183400 3 2 1
proceed [now=2025-06-06 18:51:46.969860, 2000]
found =  45 212200 2 2 1
found =  45 218500 3 2 1
found =  45 235900 2 2 1
found =  45 265500 2 2 1
found =  45 267700 3 2 1
found =  45 277300 2 2 1
found =  45 290600 2 2 1
proceed [now=2025-06-06 18:53:05.752307, 3000]
completed [now=2025-06-06 18:53:05.752470]


In [37]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=6
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 18:53:11.209322]
found =  45 2700 2 2 0
found =  45 5400 3 2 0
found =  45 17300 3 2 0
found =  45 25600 2 2 0
found =  45 28600 2 2 0
found =  45 29900 2 2 0
found =  45 34500 2 2 0
found =  45 44500 2 3 0
found =  45 51300 2 2 0
found =  45 53700 3 2 0
found =  45 55000 2 2 0
found =  45 55200 2 2 0
found =  45 58800 2 2 0
found =  45 60000 4 2 0
found =  45 64300 4 2 0
found =  45 70800 3 2 0
found =  45 74900 2 2 0
found =  45 84600 2 2 0
proceed [now=2025-06-06 18:54:28.892974, 1000]
found =  45 101600 4 2 0
found =  45 105500 2 2 0
found =  45 106800 2 2 0
found =  45 111000 2 2 0
found =  45 112600 2 2 0
found =  45 115700 2 2 0
found =  45 117300 2 2 0
found =  45 119800 3 2 0
found =  45 124500 2 2 0
found =  45 130700 2 2 0
found =  45 137200 3 2 0
found =  45 148900 3 2 0
found =  45 158900 3 2 0
found =  45 166200 2 2 0
found =  45 171600 4 2 0
found =  45 172300 2 2 0
found =  45 173500 2 2 0
found =  45 177400 4 2 0
found =  45 184100 2 2 0
found =  

In [42]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=5
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:02:12.911953]
found =  45 3500 2 2 1
found =  45 4200 4 2 1
found =  45 5100 2 2 1
found =  45 8400 3 2 1
found =  45 9400 3 2 1
found =  45 11000 2 2 1
found =  45 17100 2 2 1
found =  45 17500 3 2 1
found =  45 17800 2 2 1
found =  45 23800 4 2 1
found =  45 26200 4 2 1
found =  45 26400 3 2 1
found =  45 28900 2 2 1
found =  45 31900 3 3 1
found =  45 33700 2 2 1
found =  45 40800 2 2 1
found =  45 41500 3 2 1
found =  45 43300 3 2 1
found =  45 43900 2 2 1
found =  45 44000 2 2 1
found =  45 45400 2 2 1
found =  45 46200 2 4 1
found =  45 47300 2 2 1
found =  45 53800 3 2 1
found =  45 54200 2 2 1
found =  45 55200 2 2 1
found =  45 55500 2 2 1
found =  45 56400 2 2 1
found =  45 60700 2 2 1
found =  45 60900 2 2 1
found =  45 61400 4 2 1
found =  45 62200 3 2 1
found =  45 66800 2 2 1
found =  45 74300 3 2 1
found =  45 75900 2 2 1
found =  45 82700 2 2 1
found =  45 86000 2 2 1
found =  45 88700 3 3 1
found =  45 89100 2 2 1
found =  45 93900 2 2 1
found 

In [43]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=4
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:07:25.108848]
found =  45 1700 2 2 4
found =  45 12600 2 2 4
found =  45 14300 3 2 4
found =  45 16200 2 2 4
found =  45 32200 3 2 4
found =  45 52900 2 2 4
found =  45 75800 2 2 4
found =  45 82700 2 2 4
found =  45 83100 2 2 4
found =  45 90700 2 2 4
found =  45 91800 2 2 4
proceed [now=2025-06-06 19:08:41.740656, 1000]
found =  45 101500 2 2 4
found =  45 124400 2 2 4
found =  45 128300 3 2 4
found =  45 131900 3 2 4
found =  45 145100 3 2 4
found =  45 153700 2 2 4
found =  45 171700 2 2 4
found =  45 185100 3 2 4
found =  45 192800 2 2 4
proceed [now=2025-06-06 19:10:01.694968, 2000]
found =  45 200700 3 2 4
found =  45 203700 2 2 4
found =  45 209500 2 2 4
found =  45 210400 2 2 4
found =  45 230900 2 2 4
found =  45 236500 2 2 4
found =  45 239900 2 3 4
found =  45 242900 2 2 4
found =  45 257000 2 2 4
found =  45 265300 3 2 4
found =  45 273600 2 2 4
found =  45 278300 2 2 4
found =  45 290200 2 2 4
found =  45 294000 3 2 4
proceed [now=2025-06-06 19:11

In [47]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=3
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:12:08.601488]
found =  45 2500 2 2 3
found =  45 8100 3 2 3
found =  45 8900 2 2 3
found =  45 9700 2 2 3
found =  45 17400 2 2 3
found =  45 24300 2 2 3
found =  45 26900 3 2 3
found =  45 27300 2 2 3
found =  45 27400 2 2 3
found =  45 30900 2 2 3
found =  45 31100 2 2 3
found =  45 32000 3 2 3
found =  45 34900 2 2 3
found =  45 40400 2 2 3
found =  45 40600 2 2 3
found =  45 50900 3 2 3
found =  45 51100 2 2 3
found =  45 57000 2 2 3
found =  45 57800 2 2 3
found =  45 60900 2 2 3
found =  45 62200 2 2 3
found =  45 62600 2 2 3
found =  45 64100 2 2 3
found =  45 72000 2 2 3
found =  45 76500 2 2 3
found =  45 78700 2 2 3
found =  45 79700 2 2 3
found =  45 80000 2 2 3
found =  45 83500 2 2 3
found =  45 84700 2 2 3
found =  45 93800 3 2 3
found =  45 95400 2 2 3
found =  45 96100 2 2 3
proceed [now=2025-06-06 19:13:33.680338, 1000]
found =  45 100300 2 2 3
found =  45 101100 2 2 3
found =  45 102500 2 2 3
found =  45 104100 2 2 3
found =  45 105600 2 2 3
f

In [48]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=2
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:18:53.468890]
found =  45 3000 3 2 2
found =  45 14200 2 2 2
found =  45 14700 2 2 2
found =  45 14900 2 2 2
found =  45 16300 2 2 2
found =  45 17500 3 2 2
found =  45 18900 2 2 2
found =  45 23500 2 2 2
found =  45 30600 2 2 2
found =  45 31500 2 2 2
found =  45 34100 2 2 2
found =  45 35600 2 2 2
found =  45 38200 2 2 2
found =  45 41400 2 2 2
found =  45 46000 2 2 2
found =  45 47100 2 2 2
found =  45 48800 2 2 2
found =  45 54100 2 2 2
found =  45 64800 2 2 2
found =  45 69800 2 2 2
found =  45 75200 4 2 2
found =  45 83400 3 2 2
found =  45 83500 2 2 2
found =  45 84000 2 2 2
found =  45 85300 2 2 2
found =  45 90100 2 2 2
found =  45 93800 2 2 2
proceed [now=2025-06-06 19:20:13.362191, 1000]
found =  45 101700 4 3 2
found =  45 103400 2 2 2
found =  45 104700 3 2 2
found =  45 107100 2 2 2
found =  45 110200 2 2 2
found =  45 113300 2 2 2
found =  45 122400 3 2 2
found =  45 123100 2 2 2
found =  45 123800 2 2 2
found =  45 125300 3 2 2
found =  45 12810

In [49]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=1
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:26:02.381206]
found =  45 3200 2 2 1
found =  45 8800 2 2 1
found =  45 13100 2 2 1
found =  45 14600 2 2 1
found =  45 15400 2 2 1
found =  45 16800 2 2 1
found =  45 18700 3 3 1
found =  45 20400 3 2 1
found =  45 20700 2 2 1
found =  45 21600 2 2 1
found =  45 26600 2 2 1
found =  45 27800 3 2 1
found =  45 31900 3 2 1
found =  45 33200 2 2 1
found =  45 34600 2 2 1
found =  45 35200 3 2 1
found =  45 35300 2 2 1
found =  45 35700 3 2 1
found =  45 37100 2 2 1
found =  45 40100 2 2 1
found =  45 42000 2 3 1
found =  45 43500 2 2 1
found =  45 47500 2 2 1
found =  45 48000 3 2 1
found =  45 50200 2 2 1
found =  45 53400 3 2 1
found =  45 53900 3 2 1
found =  45 54200 3 2 1
found =  45 54500 2 2 1
found =  45 55900 3 2 1
found =  45 56700 2 2 1
found =  45 57200 2 2 1
found =  45 57400 2 2 1
found =  45 64800 2 2 1
found =  45 67300 2 2 1
found =  45 72600 2 2 1
found =  45 74200 2 2 1
found =  45 75800 3 2 1
found =  45 76300 2 2 1
found =  45 77300 2 2 1
fou

In [50]:
n_estimators=45
random_state=2000
begin_pos=0
end_pos=300000
minus_value=0
results=[]
print(f'start [now={datetime.now()}]')
cnt = 0
for i in range(begin_pos, end_pos, 100):
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=i,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state)
    if predict_len1 > 1 and predict_len2 > 1:
        results.append((n_estimators, i, predict_len1, predict_len2, minus_value))
        print('found = ', n_estimators, i, predict_len1, predict_len2, minus_value)
    cnt += 1
    if cnt % 1000 == 0:
        print(f'proceed [now={datetime.now()}, {cnt}]')        
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:31:40.345994]
found =  45 1800 2 2 0
found =  45 5000 2 2 0
found =  45 6000 2 4 0
found =  45 12800 2 2 0
found =  45 13300 2 2 0
found =  45 14300 2 2 0
found =  45 15600 2 2 0
found =  45 21200 2 2 0
found =  45 21700 2 2 0
found =  45 21900 2 2 0
found =  45 26000 2 2 0
found =  45 26200 2 2 0
found =  45 26800 2 2 0
found =  45 28600 2 2 0
found =  45 30100 2 2 0
found =  45 34500 2 2 0
found =  45 35500 3 2 0
found =  45 36000 2 2 0
found =  45 37700 2 2 0
found =  45 40200 2 2 0
found =  45 41300 2 2 0
found =  45 42300 2 2 0
found =  45 46200 2 2 0
found =  45 46800 2 2 0
found =  45 47400 2 2 0
found =  45 49600 2 2 0
found =  45 50100 2 2 0
found =  45 56600 2 2 0
found =  45 58300 2 2 0
found =  45 58600 3 3 0
found =  45 61100 2 3 0
found =  45 64200 2 2 0
found =  45 68400 2 2 0
found =  45 70900 2 2 0
found =  45 72000 2 2 0
found =  45 73400 2 2 0
found =  45 75200 2 2 0
found =  45 80700 2 2 0
found =  45 81900 2 2 0
found =  45 83000 2 2 0
foun

In [61]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[4]
    results=[]
    print(f'start [now={datetime.now()}]')
    data_pos=1
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len1=test_prediction(round_nums[data_pos],
                                 actual_numbers[data_pos],
                                 bonus[data_pos],
                                 predicted_numbers_set,
                                 n_estimators,
                                 random_state,
                                 verbose=1)
    data_pos=2
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[data_pos],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')

start [now=2025-06-06 19:53:49.764707]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14, 17, 36, 39]
일치 개수: 4
예측 번호: [14, 17, 23, 29, 36, 39]
n_estimators: 45
random_state: 208000

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [5, 20, 30]
일치 개수: 3
예측 번호: [5, 13, 20, 24, 30, 37]
n_estimators: 45
random_state: 208000
start [now=2025-06-06 19:53:49.859536]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14, 17, 36, 39]
일치 개수: 4
예측 번호: [14, 17, 23, 30, 36, 39]
n_estimators: 45
random_state: 101700

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [5, 20, 30]
일치 개수: 3
예측 번호: [5, 13, 20, 25, 30, 36]
n_estimators: 45
random_state: 101700
start [now=2025-06-06 19:53:50.016935]

--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14, 17, 36, 39]
일치 개수:

In [63]:
datas = [
    [45,208000,4,3,5],
    [45,101700,4,3,2],
    [45,185400,4,3,2],
    [45,96800,4,3,1],
    [45,58600,3,3,0],
    ]
print(f'start [now={datetime.now()}]')
results=[]
for data in datas:
    n_estimators=data[0]
    random_state=data[1]
    minus_value=data[4]    
    data_pos=0
    h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]-minus_value]
    predicted_numbers_set = predict(lotto_history_data=h_data,
                                    n_estimators=n_estimators,
                                    random_state=random_state,
                                    verbose=0)
    results.append(predicted_numbers_set[0])
    predict_len2 = test_prediction(round_nums[data_pos],
                                   actual_numbers[1],
                                   bonus[data_pos],
                                   predicted_numbers_set,
                                   n_estimators,
                                   random_state,
                                   verbose=1)
print(f'completed [now={datetime.now()}]')
print('Predicted Numbers.')
for result in results:
    print(result)

start [now=2025-06-06 19:59:26.719459]

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14]
일치 개수: 1
예측 번호: [9, 14, 24, 29, 37, 41]
n_estimators: 45
random_state: 208000

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: []
일치 개수: 0
예측 번호: [15, 19, 25, 30, 37, 41]
n_estimators: 45
random_state: 101700

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [14]
일치 개수: 1
예측 번호: [14, 18, 25, 30, 37, 41]
n_estimators: 45
random_state: 185400

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [11, 36]
일치 개수: 2
예측 번호: [11, 16, 24, 29, 36, 41]
n_estimators: 45
random_state: 96800

--- 참고: 1175회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 0

--- 예측과 실제 결과 비교 ---
일치하는 번호: [17, 36]
일치 개수: 2
예측 번호: [13, 17, 25, 30, 36, 40]
n_estimators: 45
random_state: 58600
completed [now=2025-06-06 19:59:26.918482]
Predicted 

In [32]:
print(results)

[]


```
[
[45,208000,4,3,5],
[45,101700,4,3,2],
[45,101700,4,3,2],
[45,185400,4,3,2],
[45,96800,4,3,1],
    [45,58600,3,3,0],
]
```

In [128]:
n_estimators=10
random_state=5000
data_pos=1
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0
                               )
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)
data_pos=2
h_data=lotto_history_data[r_pos[data_pos][0]:r_pos[data_pos][1]]
predicted_numbers_set = predict(lotto_history_data=h_data,
                                n_estimators=n_estimators,
                                random_state=random_state,
                                verbose=0)
test_prediction(round_nums[data_pos],
                actual_numbers[data_pos],
                bonus[data_pos],
                predicted_numbers_set,
                n_estimators,
                random_state)


--- 참고: 1174회 실제 당첨 번호 ---
당첨번호: [8, 11, 14, 17, 36, 39], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [36]
일치 개수: 1
예측 번호: [18, 19, 22, 27, 34, 36]
n_estimators: 10
random_state: 5000

--- 참고: 1173회 실제 당첨 번호 ---
당첨번호: [1, 5, 18, 20, 30, 35], 보너스: 22

--- 예측과 실제 결과 비교 ---
일치하는 번호: [18]
일치 개수: 1
예측 번호: [10, 18, 24, 28, 32, 40]
n_estimators: 10
random_state: 5000


In [ ]:
[
    ([14, 17, 23, 27, 34, 36], [14, 17, 36], (45, 1000), (1160, 1173),
    ([13, 16, 23, 27, 34, 36], [36], (45, 1300), (1160, 1173)),
    ([11, 15, 21, 30, 36, 39], [11, 36, 39], (45, 1000), (1160, 1173),

]